In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import numpy as np

# Read the JSON data into a DataFrame

# Load the data from two files
data2 = pd.read_json('trials_32.json')
data4 = pd.read_json('trials_34.json')
data3 = pd.read_json('trials_33.json')
data1 =pd.read_json('trials_31.json')
data5= pd.read_json('trials_35.json')


# # Concatenate the two dataframes
data = pd.concat([data3,data1,data2,data5,data4], ignore_index=True)
data = data[data['trial_type'] == 'main']


In [2]:
def calculate_paths(graph, start_node):
    # Recursive function to traverse the binary tree
    def traverse(node, current_path, all_paths):
        if node is None or node >= len(graph) or not graph[node]:
            # Reached a leaf node, add current path to all paths
            all_paths.append(current_path)
            return
        # Visit left and right children
        for child in graph[node]:
            traverse(child, current_path + [child], all_paths)
    
    all_paths = []
    traverse(start_node, [start_node], all_paths)
    return all_paths

# Function to calculate cumulative rewards for each path
def calculate_reward(path,rewards):
    path_rewards = []
    for path in paths:
        cumulative_reward = sum(rewards[node] for node in path if rewards[node] is not None)
        path_rewards.append(cumulative_reward)
    return path_rewards

def find_decision_path(visit_states, paths):
    """
    Determines the final decision based on visit states and paths.
    Assumes visit_states[-1] gives the final decision node.
    Returns the index of the path corresponding to the final decision.
    """
    final_decision_node = visit_states[-1]
    for i, path in enumerate(paths):
        if final_decision_node in path:
            return i + 1  # Paths are 1-indexed in decision making
    return None

def adjust_decision_based_on_path(decision2):
    """
    Adjusts decision1 based on decision2 as per the user's instructions.
    """
    if decision2 in [1, 2]:
        return 1
    elif decision2 in [3, 4]:
        return 2
    return None


In [3]:
def get_tree_structure(graph, start_node):
    first_nodes = graph[start_node] if start_node < len(graph) and graph[start_node] else []
    second_level_nodes = [graph[node] for node in first_nodes if node < len(graph) and graph[node]]
    second_nodes = sum(second_level_nodes, [])

    return first_nodes, second_nodes


def calculate_rewards(path1, path2, rewards):
    # Initialize lists to store rewards for path1 and path2
    path1_rewards = []
    path2_rewards = []

    # Calculate rewards for path1
    for node in path1:
        # Append reward for node in path1, ensure reward is not None, else append 0
        reward = rewards[node] if node < len(rewards) and rewards[node] is not None else 0
        path1_rewards.append(reward)

    # Calculate rewards for path2, which is a list of lists
    for node in path2:
        subpath_reward = rewards[node] if node < len(rewards) and rewards[node] is not None else 0
        path2_rewards.append(subpath_reward)

    # Return both lists of rewards
    return path1_rewards, path2_rewards


In [4]:
trial_1 = []
trial_2 = []
for index, trial in data.iterrows():
    graph = trial["trial"]["graph"]
    rewards = trial["trial"]["rewards"]
    start = trial["trial"]["start"]
    events = trial["events"]
    diff = trial["trial"]['difficulty']
    # Calculate paths and rewards for paths
    paths = calculate_paths(graph, start)
    path_rewards = calculate_reward(paths, rewards)
    visit_times = [event['time'] for event in trial['events'] if event['event'] == 'visit']
    # Now checking for three "visit" events to calculate the time differences
    # if len(visit_times) == 3:
    #     RT_first = visit_times[1] - visit_times[0]
    #     RT_first = visit_times[2] - visit_times[1]
    
    #Tree layer value
    path1,path2 = get_tree_structure(graph, start)
    value1,value2 = calculate_rewards(path1, path2, rewards)
    
    # Extracting visit times and states (excluding initial visit)
    visit_events = [e for e in events if e["event"] == "visit"]
    visit_states = [e["state"] for e in visit_events if not e.get("initial", False)]
    
    # Calculate RTs
    visit_times = [event['time'] for event in events if event['event'] == 'visit']
    RT_first, RT_second = None, None  # Initialize to handle cases where there are not exactly 3 visit times
    if len(visit_times) >= 3:  # Ensure there are at least 3 times recorded
        RT_first = visit_times[1] - visit_times[0]
        RT_second = visit_times[2] - visit_times[1]
    
    # Determine decision2 based on the final visit state
    decision2 = find_decision_path(visit_states, paths)
    
    # Adjust decision1 based on decision2
    decision1 = adjust_decision_based_on_path(decision2)
    if len(path_rewards) == 4:
        trial_1.append({
            "rewards": path_rewards,
            "value1":value1,
            "value2": value2,
            "choice1": decision1,
            "choice2": decision2,
            "rt1": RT_first,
            "rt2": RT_second if RT_second is not None else None,
            "difficulty": diff
        })
            
    else: 
        trial_2.append({
            "rewards": path_rewards,
            "value1":value1,
            "value2": value2,
            "choice1": decision1,
            "choice2": decision2,
            "rt1": RT_first,
            "rt2": RT_second if RT_second is not None else None,
            "RT": RT_first + RT_second
            "difficulty": diff
        }
        
            
    )



trial_1 = [trial for trial in trial_1 if trial["rt1"] is not None and 200 < trial["RT"] <= 10000 ]
trial_2 = [trial for trial in trial_2 if trial["rt1"] is not None and 200 < trial["RT"] <= 10000 ]

SyntaxError: invalid syntax (1083946486.py, line 60)

In [ ]:
# Convert the processed_trials list of dictionaries to a DataFrame
trial1_df = pd.DataFrame(trial_1)
trial2_df = pd.DataFrame(trial_2)
data_df = pd.concat([trial1_df,trial2_df], ignore_index=True)
# Export the DataFrame to a JSON file
trial1_df.to_json('trials1.json', orient='records', lines=True)
trial2_df.to_json('trials2.json', orient='records', lines=True)
data_df.to_json('AllTrial.json', orient='records', lines=True)
# Provide the path to the saved file
'trial1.json'
'trial2.json'
'AllTrial.json'